<a href="https://colab.research.google.com/github/Tachikoma-r/tec_estimate/blob/main/dcb_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/
!pip install georinex &> /dev/null
!pip install pymap3d &> /dev/null
import numpy as np
# from datetime import timedelta
import datetime
from scipy import interpolate
from pymap3d import ecef2geodetic,ecef2aer,aer2geodetic,geodetic2ecef
import math
import pymap3d as pm
import matplotlib.pyplot as plt
from pandas import Timestamp
import georinex as gr
import glob, os
from scipy.optimize import least_squares
from scipy import signal
import matplotlib.dates as md
import pandas as pd
from pandas import DataFrame
import xarray as xr
from matplotlib.dates import YearLocator, MonthLocator, DateFormatter

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [ ]:
f1 = 1575420000
f2 = 1227600000
f5 = 1176450000
c0 = 299792458


def solveIter(mu,e):

    thisStart = np.asarray(mu-1.01*e)
    thisEnd = np.asarray(mu + 1.01*e)
    bestGuess = np.zeros(mu.shape)

    for i in range(5): 
        minErr = 10000*np.ones(mu.shape)
        for j in range(5):
            thisGuess = thisStart + j*(thisEnd-thisStart)/10.0
            thisErr = np.asarray(abs(mu - thisGuess + e*np.sin(thisGuess)))
            mask = thisErr<minErr
            minErr[mask] = thisErr[mask]
            bestGuess[mask] = thisGuess[mask]
        
        # reset for next loop
        thisRange = thisEnd - thisStart
        thisStart = bestGuess - thisRange/10.0
        thisEnd = bestGuess + thisRange/10.0
        
    return(bestGuess)


def getGpsTime(dt):
    """_getGpsTime returns gps time (seconds since midnight Sat/Sun) for a datetime
    """
    total = 0
    days = (dt.weekday()+ 1) % 7 
    total += days*3600*24
    total += dt.hour * 3600
    total += dt.minute * 60
    total += dt.second
    return(total)

def gpsSatPosition(fnav, dt, sv=None, rx_position=None, coords='xyz'):
    navdata = gr.load(fnav).sel(sv=sv)
    timesarray = np.asarray(dt,dtype='datetime64[ns]') 
    navtimes = navdata.time.values 
    idnan = np.isfinite(navdata['Toe'].values)
    navtimes = navtimes[idnan]
    bestephind = []
    for t in timesarray:
        idt = abs(navtimes - t).argmin()
        bestephind.append(idt)
    gpstime = np.array([getGpsTime(t) for t in dt])
    t = gpstime - navdata['Toe'][idnan][bestephind].values 

    GM = 3986005.0E8 
    OeDOT = 7.2921151467E-5

    ecc = navdata['Eccentricity'][idnan][bestephind].values # Eccentricity
    Mk = navdata['M0'][idnan][bestephind].values + \
         t *(np.sqrt(GM / navdata['sqrtA'][idnan][bestephind].values**6) + 
             navdata['DeltaN'][idnan][bestephind].values)
    Ek = solveIter(Mk,ecc)
    Vk = np.arctan2(np.sqrt(1.0 - ecc**2) * np.sin(Ek), np.cos(Ek) - ecc)
    PhiK = Vk + navdata['omega'][idnan][bestephind].values
    # Perturbations
    delta_uk = navdata['Cuc'][idnan][bestephind].values * np.cos(2.0*PhiK) + \
              navdata['Cus'][idnan][bestephind].values * np.sin(2.0*PhiK)
    Uk = PhiK + delta_uk
    
    delta_rk = navdata['Crc'][idnan][bestephind].values * np.cos(2.0*PhiK) + \
               navdata['Crs'][idnan][bestephind].values * np.sin(2.0*PhiK)
    Rk = navdata['sqrtA'][idnan][bestephind].values**2 * (1.0 - ecc * np.cos(Ek)) + delta_rk
    
    delta_ik = navdata['Cic'][idnan][bestephind].values * np.cos(2.0*PhiK) + \
               navdata['Cis'][idnan][bestephind].values * np.sin(2.0*PhiK)
    Ik = navdata['Io'][idnan][bestephind].values + \
         navdata['IDOT'][idnan][bestephind].values * t + delta_ik

    Omegak = navdata['Omega0'][idnan][bestephind].values + \
             (navdata['OmegaDot'][idnan][bestephind].values - OeDOT) * t - \
             (OeDOT * navdata['Toe'][idnan][bestephind].values)
             
    #X,Y coordinate corrections
    Xkprime = Rk * np.cos(Uk)
    Ykprime = Rk * np.sin(Uk)
    # ECEF XYZ
    X = Xkprime * np.cos(Omegak) - (Ykprime * np.sin(Omegak) * np.cos(Ik))
    Y = Xkprime * np.sin(Omegak) + (Ykprime * np.cos(Omegak) * np.cos(Ik))
    Z = Ykprime * np.sin(Ik)
    
    if coords == 'xyz':
        return np.array([X,Y,Z])
    elif coords == 'aer':
        assert rx_position is not None
        rec_lat, rec_lon, rec_alt = ecef2geodetic(rx_position[0], rx_position[1], rx_position[2])
        A,E,R = ecef2aer(X, Y, Z, rec_lat, rec_lon, rec_alt)
        return np.array([A,E,R])

In [ ]:
conversionFactor = 0.463*6.158

def getIntervals(L1, L2, P1, P2, maxgap=3,maxjump=2):
    """
    Greg Starr
    scans through the phase tec of a satellite and determines where "good"
    intervals begin and end
    inputs:
        L1, L2, P1, P2 - np arrays of the raw data used to calculate phase corrected
        TEC. 
        maxgap - maximum number of nans before starting new interval
        maxjump - maximum jump in phase TEC before starting new interval
    output:
        intervals - list of 2-tuples, beginning and end of each "good" interval
                    as a Pandas/numpy
    """
    global f1
    global f2
    
    r = np.array(range(len(P1)))
    idx = np.isfinite(L1) & np.isfinite(L2) & np.isfinite(P1) & np.isfinite(P2)
    r = r[idx]
    intervals=[]
    if len(r)==0:
        return idx, intervals
    phase_tec=2.85E9*(L1/f1-L2/f2)
    beginning=r[0]
    last=r[0]
    for i in r[1:]:
        if (i-last>maxgap) or (abs(phase_tec[i]-phase_tec[last])>maxjump):
            intervals.append((beginning,last))
            beginning=i
        last=i
        if i==r[-1]:
            intervals.append((beginning,last))
    return idx, intervals


def slantTEC(C1, C2, L1, L2, f):
    global f1, c0
    F = (f1**2 * f**2) / (f1**2 - f**2)
    rangetec = F / 40.3 * (C2 - C1) / 1e16
    phasetec = F / 40.3 * c0 * (L1/f1 - L2/f) / 1e16
    N = np.nanmedian(rangetec - phasetec)
    return phasetec + N

def dataFromNC(fnc,fnav, sv, fsp3=None,
               el_mask=30,tlim=None, 
               satpos=False,
               ipp=False,
               ipp_alt=None):
  
    D = gr.load(fnc, useindicators=True).sel(sv=sv)
    D['time'] = np.array([np.datetime64(ttt) for ttt in D.time.values])
    if tlim is not None:
        if len(tlim) == 2:
            t0 = tlim[0] 
            t1 = tlim[1] 
            D = D.where(np.logical_and(D.time >= np.datetime64(t0), 
                                       D.time <= np.datetime64(t1)), 
                                       drop=True)
    obstimes64 = D.time.values
    dt = np.array([Timestamp(t).to_pydatetime() for t in obstimes64])
    aer = gpsSatPosition(fnav, dt, sv=sv, rx_position=D.position, coords='aer')
    idel = (aer[1,:] >= el_mask)
    aer[:,~idel] = np.nan
    D['time'] = dt

    if satpos:
        D['az'] = aer[0,:]
        D['el'] = aer[1,:]
        D['idel'] = idel
    if ipp:
        if ipp_alt is None:
            print ('Auto assigned altitude of the IPP: 250 km')
            ipp_alt = 250e3
        else:
            ipp_alt *= 1e3
        rec_lat, rec_lon, rec_alt = D.position_geodetic
        fm = np.sin(np.radians(aer[1]))
        r_new = ipp_alt / fm
        lla_vector = np.array(aer2geodetic(aer[0], aer[1], r_new, rec_lat, rec_lon, rec_alt))
        D['ipp_lon'] = lla_vector[1]
        D['ipp_lat'] = lla_vector[0]
        D['ipp_alt'] = ipp_alt
    return D

def getMappingFunction(elevation, H):
    rc1 = 6371.0 / (6371.0 + H)
    f = np.sqrt(1 - (np.cos(np.radians(elevation))**2 * rc1**2))
    return f

def read_one_sinex(file):
    import pandas as pd
    df = pd.read_fwf(file, skiprows=57)
    df.drop(df.tail(2).index, inplace=True) 
    df.columns = ['bias', 'svn', 'prn', 'station', 'obs1', 'obs2',
                  'bias_start', 'bias_end', 'unit', 'value', 'std']
    ds = xr.Dataset()
    ds.attrs['bias'] = df['bias'].values[0]
    df_sat = df[df['station'].isnull()]
    df_station = df[~df['station'].isnull()]
    return df


In [ ]:
obspath = '/content/drive/MyDrive/zimm0970.22d.gz'
navpath = '/content/drive/MyDrive/zimm0970.22n.gz'
sinexfile = "CAS0MGXRAP_20220970000_01D_01D_DCB.BSX"

NCF = obspath
NF  = navpath

sx = read_one_sinex(sinexfile)
df_sat = sx[sx['station'].isnull()]
df_station = sx[~sx['station'].isnull()]
recv_name = ['ZIMM']
bias_values = ['C1C', 'C2C', 'C1W', 'C2W']
recvbias = sx[~sx['station'].isnull()].loc[(sx['station']==recv_name[0]) & (sx['obs1'] == bias_values[0]) & (sx['obs2'] == bias_values[3]) ].value.values

SB = True
el_mask = 30
tlim = None
H = 350
finalresult = ''

svlist = gr.load(NCF).sv.values
navdata = gr.load(NF)
navdatatime = navdata.time.values
obs = gr.load(NCF)
time = obs.time.values


stec = np.nan * np.ones((svlist.shape[0], obs.time.values.shape[0]))
F = np.nan * np.ones((svlist.shape[0], obs.time.values.shape[0]))
vtec = np.nan * np.ones((svlist.shape[0], obs.time.values.shape[0]))
for i, sv in enumerate(svlist[:10]):
    D = dataFromNC(NCF, NF, sv=sv, tlim=tlim, el_mask=el_mask, satpos=True, 
                          ipp=True, ipp_alt = H)
    idel = D['idel'].values
    satbias = sx.loc[(sx['prn'] == sv) & (sx['obs1'] == bias_values[0]) & (sx['obs2'] == bias_values[3])].value.values
    bias = (recvbias + satbias) * conversionFactor
    dt = D.time.values
    tsps = np.diff(dt.astype('datetime64[s]'))[0].astype(int)
    el = D.el.values
    az = D.az.values
    lat, lon = D.ipp_lat.values, D.ipp_lon.values
    C1 = D['C1'].values
    C1[~idel] = np.nan
    C2 = D['P2'].values
    C2[~idel] = np.nan
    L1 = D['L1'].values
    L1[~idel] = np.nan
    L2 = D['L2'].values
    L2[~idel] = np.nan
    
    ixin, intervals = getIntervals(L1,L2,C1,C2, maxgap=1)
    tec = np.nan * stec[0]
    for r in intervals:
        tec[r[0]:r[-1]] = slantTEC(C1[r[0]:r[-1]], C2[r[0]:r[-1]], 
                                          L1[r[0]:r[-1]], L2[r[0]:r[-1]],f2)
    stec[i, :] = tec + bias
    F[i, :] = getMappingFunction(el, H)
    
    ts = pd.to_datetime(dt) 
    d = ts.strftime('%H:%M:%S')
    vtec[i , :] = stec[i, :]*F[i, :]

    for k, tt in enumerate(d):
      result = (str(tt)+","+sv+","+str(az[k])+","+str(el[k])+","+str(lat[k])+","+str(lon[k])+","+str(stec[i, k])+","+str(vtec[i, k])+"\n")
      finalresult = finalresult+result
      # result = (sv+","+str(stec[i, :])+","+str(el)+","+str(az)+","+str(lat)+","+str(lon)+"\n")
      # finalresult = finalresult+result
# print(finalresult)
# vtec = stec * F

dirpath = '/content/drive/MyDrive'
with open(dirpath + '/TECValues.csv', 'w') as csvfile:
    csvfile.write(finalresult)

df = pd.read_csv('TECValues.csv', header=None)
df.rename(columns={0: 'time', 1: 'PRN', 2: 'az', 3: 'ele', 4: 'lat', 5: 'lon', 6: 'stec', 7: 'vtec' }, inplace=True)
df.to_csv('TECV.csv', index=False) # save to new csv file

# fig1 = plt.figure(figsize = [8,6])
# ax1 = fig1.add_subplot(111)
# for y in vtec:
#     ax1.plot(time, y)



In [ ]:
# data_vars = {'C1':(['time', 'sv'], obs.C1)}
# coords = {'time': (['time'], obs.time.values),'sv': (['sv'], obs.sv.values)}
# ds = xr.Dataset(data_vars=data_vars, coords=coords)

ds = xr.Dataset(
        data_vars={'C1':    (('time', 'sv'), obs.C1.values),
                   'C2': (('time', 'sv'), obs.C2.values),
                   'P1':    (('time', 'sv'), obs.C1.values)},
        coords={'time': obs.time.values,
                'sv': obs.sv.values})

In [ ]:
ds.to_netcdf('process.nc')
ndf = ds.to_netcdf()

In [ ]:
import h5py
f = h5py.File('process.nc', 'r')
data = f['C1'][:]
f.close()
print(data[:])

[[         nan 24074430.078 22686044.766 ...          nan          nan
           nan]
 [         nan 24059464.5   22701845.391 ...          nan          nan
           nan]
 [         nan 24044553.266 22717685.492 ...          nan          nan
           nan]
 ...
 [         nan 23996316.07  22769713.625 ...          nan          nan
           nan]
 [         nan 23981635.258 22785717.828 ...          nan          nan
           nan]
 [         nan 23967010.469 22801760.828 ... 25431279.164          nan
           nan]]


In [ ]:
obs_t = gr.load("/content/drive/MyDrive/bara1210.22o")

IndexError: ignored

In [ ]:
obs_t

In [ ]:
os.chdir(dirpath)
ss = 22
navfiles = []
for nav in glob.glob("*."+str(ss)+"n"):
  if (str(ss)+"n") in nav:
    navfiles.append(dirpath+'/'+nav)
navfiles
# obsfiles = glob.glob(dirpath + '/*.' + 'd.Z')

['/content/drive/MyDrive/bere1210.22n', '/content/drive/MyDrive/bara1210.22n']

In [ ]:
obsfiles

[]

In [ ]:
def process():
  def sub(gap, dirpath):
    navfiles = glob.glob(dirpath + '\\*.' + stringdate[2:4] + 'n.Z')
    # obsfiles = glob.glob(dirpath + '\\*.' + stringdate[2:4] + 'd.Z')